In [1]:
from tree_construct import *
from likelihood_calculation import *
from pysam import VariantFile, FastaFile
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# import argparse
# parser = argparse.ArgumentParser(description='Calculation of haplogroups')
# parser.add_argument('tree', type=str, help='haplogroup tree')
# parser.add_argument('ref', type=str, help='reference fasta')
# parser.add_argument('vcf', type=str, help='vcf file of mtdna')
# args = parser.parse_args()


# with open(args.tree) as f:
#     d = json.load(f) # d - это список python
# #     print(d)
# for i in d:
#     i[0] += 1
#     d[0][0]=0

# bcf_in = VariantFile(args.vcf) 
# ref = FastaFile(args.ref)

In [3]:
def glhap(bcf_in_str, comment=True):
    make_tree('array/array.json')
    ref = FastaFile('refchrm.fa')
    bcf_in = VariantFile(bcf_in_str) 
    
    ref_lh = calculate_likelihood(bcf_in, ref)
    gls = get_log_monozygous(bcf_in)
    
    prunung(a, ref, ref_lh, gls)
    
    key = lambda x: x.lh
    S = list()
    for i in PreOrderIter(a):
        S.append(i)
        
    S.sort(key = key,reverse=False)
    if comment:
        for i in range(10):
            print(S[i].name+",lh = ",S[i].lh)
    
    # print(ref_lh)
    return [(S[i].name, S[i].lh) for i in range(10)]

In [4]:
a = make_tree('array/array.json')

In [10]:
for i in PreOrderIter(a):
    if len(i.insertion) > 0:
        break
print(i)
i.insertion

L0d1d


{2491}

In [11]:
glhap("in1.vcf.gz")

X2b5,lh =  6122
-,lh =  6632
X2b,lh =  6887
X2b4,lh =  6887
X2b7,lh =  6887
X2b8,lh =  6887
X2b9,lh =  6887
-,lh =  6887
X2b1,lh =  7142
X2b3,lh =  7142


[('X2b5', 6122),
 ('-', 6632),
 ('X2b', 6887),
 ('X2b4', 6887),
 ('X2b7', 6887),
 ('X2b8', 6887),
 ('X2b9', 6887),
 ('-', 6887),
 ('X2b1', 7142),
 ('X2b3', 7142)]

In [ ]:
# INSERTIONS
# insertions = []
# for rec in bcf_in.fetch():
#     if (rec.info['INDEL']== True):
# #         if len(rec.ref)== 1 and len(rec.alts[0]) == 2:
#             insertions.append([rec.pos + 1, -rec.samples.values()[0]['PL'][-1]/10])

In [ ]:
# Deletions
# deletions = []
# for rec in bcf_in.fetch():
#     if (rec.info['INDEL']== True):
#         if len(rec.ref)== 2 and rec.ref[-1]!='N' and len(rec.alts[0]) == 1:
#             deletions.append([rec.pos + 1, -rec.samples.values()[0]['PL'][-1]/10])

In [ ]:
def glhap_aposterior(bcf_in_str):
    with open('array/array.json') as f:
        d = json.load(f) # d - это список python
    for i in d:
        i[0] += 1
    d[0][0]=0

    bcf_in = VariantFile(bcf_in_str) 
    ref = FastaFile('refchrm.fa')
    a = Node(d[0][1],[])
    make_tree(d,a,0)
    ref_lh = calculate_likelihood(bcf_in, ref)
    gls = get_log_monozygous(bcf_in)
    prunung(a, ref, ref_lh, gls)
    key = lambda x: x.lh
    S = list()
    for i in PreOrderIter(a):
        S.append(i)
    S.sort(key = key,reverse=True)
    
    aposterior_divisor = sum([2**S[i].lh for i in range(len(S))])
    
    return [2**S[i].lh/aposterior_divisor for i in range(10)]
    
